In [1]:
import vtk
from vtk.util import numpy_support
import numpy as np

def inspect_vti_complete(filename):
    """Complete inspection of VTI file contents"""
    print(f"\n{'='*60}")
    print(f"COMPLETE VTI FILE INSPECTION: {filename}")
    print(f"{'='*60}")
    
    # Read the file
    reader = vtk.vtkXMLImageDataReader()
    reader.SetFileName(filename)
    reader.Update()
    
    # Get the data
    data = reader.GetOutput()
    
    # 1. BASIC INFORMATION
    print("\n1. BASIC INFORMATION:")
    print("-"*40)
    dims = data.GetDimensions()
    print(f"Dimensions: {dims[0]} x {dims[1]} x {dims[2]}")
    print(f"Number of points: {data.GetNumberOfPoints()}")
    print(f"Number of cells: {data.GetNumberOfCells()}")
    
    # 2. GEOMETRY INFORMATION
    print("\n2. GEOMETRY INFORMATION:")
    print("-"*40)
    origin = data.GetOrigin()
    spacing = data.GetSpacing()
    bounds = data.GetBounds()
    print(f"Origin: ({origin[0]}, {origin[1]}, {origin[2]})")
    print(f"Spacing: ({spacing[0]}, {spacing[1]}, {spacing[2]})")
    print(f"Bounds: X[{bounds[0]}, {bounds[1]}], Y[{bounds[2]}, {bounds[3]}], Z[{bounds[4]}, {bounds[5]}]")
    extent = data.GetExtent()
    print(f"Extent: ({extent[0]}, {extent[1]}, {extent[2]}, {extent[3]}, {extent[4]}, {extent[5]})")
    
    # 3. POINT DATA
    print("\n3. POINT DATA ARRAYS:")
    print("-"*40)
    point_data = data.GetPointData()
    n_point_arrays = point_data.GetNumberOfArrays()
    print(f"Number of point data arrays: {n_point_arrays}")
    
    if n_point_arrays == 0:
        print("  No point data arrays found!")
    else:
        for i in range(n_point_arrays):
            array = point_data.GetArray(i)
            print(f"\n  Array {i+1}: '{array.GetName()}'")
            print(f"    - Number of tuples: {array.GetNumberOfTuples()}")
            print(f"    - Number of components: {array.GetNumberOfComponents()}")
            print(f"    - Data type: {array.GetDataTypeAsString()}")
            
            # Get numpy array
            np_array = numpy_support.vtk_to_numpy(array)
            print(f"    - Shape: {np_array.shape}")
            print(f"    - dtype: {np_array.dtype}")
            
            # Statistics
            if array.GetNumberOfComponents() == 1:
                print(f"    - Range: [{array.GetRange()[0]}, {array.GetRange()[1]}]")
                print(f"    - Mean: {np.mean(np_array):.6f}")
                print(f"    - Std: {np.std(np_array):.6f}")
                
                # Unique values
                unique = np.unique(np_array)
                if len(unique) <= 20:
                    print(f"    - Unique values ({len(unique)}): {unique}")
                else:
                    print(f"    - Number of unique values: {len(unique)}")
                    print(f"    - First 10 unique: {unique[:10]}")
            else:
                for j in range(array.GetNumberOfComponents()):
                    comp_range = array.GetRange(j)
                    print(f"    - Component {j} range: [{comp_range[0]}, {comp_range[1]}]")
            
            # First few values
            print(f"    - First 5 values: {np_array[:5].tolist()}")
    
    # Check if any array is set as scalars
    if point_data.GetScalars():
        print(f"\n  Active scalars: '{point_data.GetScalars().GetName()}'")
    
    # Check if any array is set as vectors
    if point_data.GetVectors():
        print(f"  Active vectors: '{point_data.GetVectors().GetName()}'")
    
    # 4. CELL DATA
    print("\n4. CELL DATA ARRAYS:")
    print("-"*40)
    cell_data = data.GetCellData()
    n_cell_arrays = cell_data.GetNumberOfArrays()
    print(f"Number of cell data arrays: {n_cell_arrays}")
    
    if n_cell_arrays == 0:
        print("  No cell data arrays found!")
    else:
        for i in range(n_cell_arrays):
            array = cell_data.GetArray(i)
            print(f"\n  Array {i+1}: '{array.GetName()}'")
            print(f"    - Number of tuples: {array.GetNumberOfTuples()}")
            print(f"    - Number of components: {array.GetNumberOfComponents()}")
            print(f"    - Data type: {array.GetDataTypeAsString()}")
            
            # Get numpy array
            np_array = numpy_support.vtk_to_numpy(array)
            print(f"    - Shape: {np_array.shape}")
            
            if array.GetNumberOfComponents() == 1:
                print(f"    - Range: [{array.GetRange()[0]}, {array.GetRange()[1]}]")
                unique = np.unique(np_array)
                if len(unique) <= 20:
                    print(f"    - Unique values ({len(unique)}): {unique}")
                else:
                    print(f"    - Number of unique values: {len(unique)}")
    
    # 5. FIELD DATA
    print("\n5. FIELD DATA:")
    print("-"*40)
    field_data = data.GetFieldData()
    n_field_arrays = field_data.GetNumberOfArrays()
    print(f"Number of field data arrays: {n_field_arrays}")
    
    if n_field_arrays > 0:
        for i in range(n_field_arrays):
            array = field_data.GetArray(i)
            if array is not None:
                print(f"\n  Array {i+1}: '{array.GetName()}'")
                print(f"    - Number of tuples: {array.GetNumberOfTuples()}")
                print(f"    - Number of components: {array.GetNumberOfComponents()}")
            else:
                print(f"\n  Array {i+1}: None")
    
    # 6. DATA OBJECT INFORMATION
    print("\n6. DATA OBJECT INFORMATION:")
    print("-"*40)
    print(f"Data object type: {data.GetClassName()}")
    print(f"Memory size: {data.GetActualMemorySize()} KB")
    
    # 7. CHECK FOR SPECIFIC COMMON ARRAYS
    print("\n7. CHECKING FOR COMMON ARRAY TYPES:")
    print("-"*40)
    common_names = {
        'grain': ['grain', 'grainid', 'grain_id', 'grainids', 'feature', 'featureid'],
        'orientation': ['euler', 'orientation', 'rodrigues', 'quaternion', 'phi1', 'phi', 'phi2'],
        'quality': ['quality', 'iq', 'ci', 'confidence'],
        'phase': ['phase', 'phaseid', 'phase_id', 'material'],
        'deformation': ['strain', 'stress', 'deformation', 'kam', 'gnd', 'energy']
    }
    
    all_array_names = []
    for i in range(point_data.GetNumberOfArrays()):
        all_array_names.append(point_data.GetArrayName(i).lower())
    for i in range(cell_data.GetNumberOfArrays()):
        all_array_names.append(cell_data.GetArrayName(i).lower())
    
    for category, keywords in common_names.items():
        found = []
        for name in all_array_names:
            for keyword in keywords:
                if keyword in name:
                    found.append(name)
                    break
        if found:
            print(f"  {category.upper()} arrays found: {found}")
        else:
            print(f"  {category.upper()} arrays: None found")
    
    print(f"\n{'='*60}")
    print("END OF VTI FILE INSPECTION")
    print(f"{'='*60}\n")

# Run the inspection
inspect_vti_complete("3grains_inc0.vti")


COMPLETE VTI FILE INSPECTION: 3grains_inc0.vti

1. BASIC INFORMATION:
----------------------------------------
Dimensions: 11 x 11 x 11
Number of points: 1331
Number of cells: 1000

2. GEOMETRY INFORMATION:
----------------------------------------
Origin: (0.0, 0.0, 0.0)
Spacing: (0.0001, 0.0001, 0.0001)
Bounds: X[0.0, 0.001], Y[0.0, 0.001], Z[0.0, 0.001]
Extent: (0, 10, 0, 10, 0, 10)

3. POINT DATA ARRAYS:
----------------------------------------
Number of point data arrays: 1

  Array 1: 'u'
    - Number of tuples: 1331
    - Number of components: 3
    - Data type: float
    - Shape: (1331, 3)
    - dtype: float32
    - Component 0 range: [0.0, 0.0]
    - Component 1 range: [0.0, 0.0]
    - Component 2 range: [0.0, 0.0]
    - First 5 values: [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0]]

4. CELL DATA ARRAYS:
----------------------------------------
Number of cell data arrays: 11

  Array 1: 'phase/mechanical/F / 1'
    - Number of tuples: 100